## 177 Null Count

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error
from mv_wg_avg import exp_weight_avg


key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [25]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
sys.exit()
df_hist = utils.read_df_pkl('../input/histori*0*')
df_new = utils.read_df_pkl('../input/new_mer*0*')

df_train = utils.reduce_mem_usage(df_train)
df_test  = utils.reduce_mem_usage(df_test )
df_hist  = utils.reduce_mem_usage(df_hist )
# df_new   = utils.reduce_mem_usage(df_new  )

df_org_lag = utils.read_pkl_gzip('../input/train_test_original_month_lag.gz')
df_org_lag = utils.reduce_mem_usage(df_org_lag)
df_org_lag.head()

df_org_lag.rename(columns={'purchase_date':'yyyymm'}, inplace=True)


df_hist['yyyymm'] =  df_hist['purchase_date'].map(lambda x: str(x)[:7])
# df_new['yyyymm'] =  df_new['purchase_date'].map(lambda x: str(x)[:7])
print(df_hist.shape)
df_hist = df_hist.merge(df_org_lag.reset_index()[[key, 'yyyymm', 'org_month_lag']], how='inner', on=[key, 'yyyymm'])
print(df_hist.shape)
# df_new = df_new.merge(df_org_lag.reset_index()[[key, 'yyyymm', 'org_month_lag']], how='inner', on=[key, 'yyyymm'])
# df_new = []

100%|██████████| 3/3 [00:00<00:00, 113.47it/s]


SystemExit: 

In [7]:
df = df_new
df.sort_values(by=[key, 'purchase_date'], inplace=True)
df['authorized_flag'] =  df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
df_new = df.copy()
del df
gc.collect()

df = df_hist
df['authorized_flag'] =  df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
df['category_1'] =  df['category_1'].map(lambda x: 0 if x=='N' else 1)
df_hist = df.copy()
del df
gc.collect()
df_hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,yyyymm,org_month_lag
0,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,2017-06,12
1,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_979ed661fc,-8,-0.737967,2017-06-01 22:02:56,1.0,16,37,2017-06,12
2,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-8,-0.741648,2017-06-08 18:02:29,1.0,16,37,2017-06,12
3,1,C_ID_4e6213e9bc,88,0,0,A,560,M_ID_e6d5ae8ea6,-8,-0.738192,2017-06-14 07:40:48,1.0,16,34,2017-06,12
4,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_979ed661fc,-8,-0.726772,2017-06-23 21:43:17,1.0,16,37,2017-06,12


In [10]:
# 期間を絞る
base_month = 0
last_month = 2
col_month = 'org_month_lag'
df = df_hist[df_hist[col_month]<=last_month]
df = df[df[col_month]>=base_month]

auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]

cat1_0 = False
# cat1_0 = True
if cat1_0:
    auth1_cat1 = auth1[auth1.category_1==1]
    auth1_cat0 = auth1[auth1.category_1==0]
    auth0_cat1 = auth0[auth0.category_1==1]
    auth0_cat0 = auth0[auth0.category_1==0]
#     del auth1, auth0
#     gc.collect()

In [5]:
auth1[auth1.isnull().any(axis=1)].head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
11,1,C_ID_4e6213e9bc,-1,0,0,A,560,M_ID_7c5e93af2f,0,-0.727929,2018-02-08 20:05:45,NaN,-1,34
30,1,C_ID_4e6213e9bc,-1,0,0,A,80,M_ID_98b342c0e3,0,-0.724368,2018-02-27 09:04:48,NaN,-1,37
43,1,C_ID_4e6213e9bc,-1,0,0,A,278,NaN,0,-0.719154,2018-02-24 10:52:32,NaN,-1,37
80,1,C_ID_4e6213e9bc,-1,0,0,A,80,M_ID_98b342c0e3,0,-0.724368,2018-02-23 09:20:59,NaN,-1,37
82,1,C_ID_4e6213e9bc,-1,0,0,A,278,NaN,0,-0.713173,2018-02-25 12:07:37,NaN,-1,37


In [11]:
cnt_cols = ['city_id', 'state_id', 'subsector_id', 'installments']
cnt_cols = ['merchant_category_id']
for col in cnt_cols:
    auth1[col] =  auth1[col].replace(-1, np.nan)
    auth0[col] =  auth0[col].replace(-1, np.nan)
    df_new[col] =  df_new[col].replace(-1, np.nan)

### Null Count

In [26]:
for fname, df in zip(['auth1', 'auth0', 'new'], [auth1, auth0, df_new]):
    tmp = df[df.isnull().any(axis=1)]
    df_nan = pd.concat([tmp[key], (~(tmp==tmp)).sum(axis=1)], axis=1).rename(columns={0:f'{fname}_null_cnt'})
    df_nan = df_nan.groupby(key)[f'{fname}_null_cnt'].sum()
    train_test = train_test.join(df_nan)
    
null_cols = [col for col in train_test.columns if col.count('null_')]
train_test[null_cols].fillna(0, inplace=True)
train_test['all_null_cnt'] = train_test[null_cols].sum(axis=1)
# df_train.drop(null_cols, axis=1, inplace=True)
# train_test.fillna(-1, inplace=True)
train_test.head()

,feature_1,feature_2,feature_3,first_active_month,target,auth1_null_cnt,auth0_null_cnt,new_null_cnt,all_null_cnt
card_id,,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06,-0.820283,NaN,NaN,NaN,0.0
C_ID_3d0044924f,4,1,0,2017-01,0.392913,19.0,NaN,NaN,19.0
C_ID_d639edf6cd,2,2,0,2016-08,0.688056,NaN,NaN,NaN,0.0
C_ID_186d6a6901,4,3,0,2017-09,0.142495,6.0,NaN,1.0,7.0
C_ID_cdbd2c0db2,1,3,0,2017-11,-0.159749,11.0,NaN,1.0,12.0


In [27]:
# 個別featureのNULLカウント
cnt_cols = ['category_2', 'category_3', 'merchant_category_id', 'city_id', 'state_id', 'subsector_id', 'installments']
for fname, df in zip(['auth1', 'auth0', 'new'],[auth1, auth0, df_new]):
    for col in cnt_cols:
        tmp = df[[key, col]]
        tmp = tmp[tmp.isnull().any(axis=1)]
        df_nan = pd.concat([tmp[key], (~(tmp==tmp)).sum(axis=1)], axis=1).rename(columns={0:f'{fname}_{col}_null_cnt'})
        df_nan = df_nan.groupby(key)[f'{fname}_{col}_null_cnt'].sum()
        train_test = train_test.join(df_nan)
    
null_cols = [col for col in train_test.columns if col.count('null_')]
train_test[null_cols].fillna(0, inplace=True)
# train_test['all_null_cnt'] = train_test[null_cols].sum(axis=1)
# train_test.drop(null_cols, axis=1, inplace=True)
# train_test.fillna(-1, inplace=True)
train_test.head()

,feature_1,feature_2,feature_3,first_active_month,target,auth1_null_cnt,auth0_null_cnt,new_null_cnt,all_null_cnt,auth1_category_2_null_cnt,auth1_category_3_null_cnt,auth1_merchant_category_id_null_cnt,auth1_city_id_null_cnt,auth1_state_id_null_cnt,auth1_subsector_id_null_cnt,auth1_installments_null_cnt,auth0_category_2_null_cnt,auth0_category_3_null_cnt,auth0_merchant_category_id_null_cnt,auth0_city_id_null_cnt,auth0_state_id_null_cnt,auth0_subsector_id_null_cnt,auth0_installments_null_cnt,new_category_2_null_cnt,new_category_3_null_cnt,new_merchant_category_id_null_cnt,new_city_id_null_cnt,new_state_id_null_cnt,new_subsector_id_null_cnt,new_installments_null_cnt
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06,-0.820283,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_ID_3d0044924f,4,1,0,2017-01,0.392913,19.0,NaN,NaN,19.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_ID_d639edf6cd,2,2,0,2016-08,0.688056,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_ID_186d6a6901,4,3,0,2017-09,0.142495,6.0,NaN,1.0,7.0,5.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
C_ID_cdbd2c0db2,1,3,0,2017-11,-0.159749,11.0,NaN,1.0,12.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [10]:
train_test['category_2_city_diff_null_cnt'] = train_test['auth1_category_2_null_cnt'] - train_test['auth1_city_id_null_cnt']

null_cols = [col for col in train_test.columns if col.count('null_') and ( col.count('city') or col.count('category_3') or col.count('merchant') or col.count('install') )]
train_test['null_cnt'] = train_test[null_cols].sum(axis=1)

null_cols = [col for col in train_test.columns if col.count('auth1_') and ( col.count('city') or col.count('category_3') or col.count('merchant') or col.count('install') )]
train_test['auth1_null_cnt'] = train_test[null_cols].sum(axis=1)

null_cols = [col for col in train_test.columns if col.count('auth0_') and ( col.count('city') or col.count('category_3') or col.count('merchant') or col.count('install') )]
train_test['auth0_null_cnt'] = train_test[null_cols].sum(axis=1)

null_cols = [col for col in train_test.columns if col.count('new_') and ( col.count('city') or col.count('category_3') or col.count('merchant') or col.count('install') )]
train_test['new_null_cnt'] = train_test[null_cols].sum(axis=1)

SystemExit: 

### Save feature

In [29]:
fname = '177_l02'
ignore_features = ['first_active_month', 'card_id', target]
df_feat = train_test

for col in df_feat.columns:
    if (col.count('state')):continue
    if (col.count('subsec')):continue
#     if (col.count('all')):continue
    if (col.count('category_2_null')):continue
    if not(col.count('null_')):continue
    length = len(df_feat[col].dropna())
    if length/len(df_feat)<0.1:
        print(col)
        continue

    df_feat[col].fillna(0, inplace=True)
    feature = df_feat[col].values.astype('float32')
    if feature.shape[0]!=325540:
        print(feature.shape)
#     utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)
    utils.to_pkl_gzip(path = f'../features/2_second_valid/{fname}_{col}@', obj=feature)

auth1_category_3_null_cnt
auth1_merchant_category_id_null_cnt
auth1_city_id_null_cnt
auth1_installments_null_cnt
auth0_category_3_null_cnt
auth0_merchant_category_id_null_cnt
auth0_city_id_null_cnt
auth0_installments_null_cnt
new_category_3_null_cnt
new_merchant_category_id_null_cnt
new_city_id_null_cnt
